In [1]:
import cv2
import numpy as np

In [2]:
video_path = "denis_walk.avi"
bg_img_path = "bg.png"
output_path = "final_output.mp4"

In [3]:
bg_img = cv2.imread(bg_img_path)
frame_size = (640, 360)
bg_img = cv2.resize(bg_img, frame_size)

In [4]:
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

In [5]:
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=25, detectShadows=False)

In [6]:
for _ in range(30):
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, frame_size)
    fgbg.apply(frame)


cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

True

In [7]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, frame_size)
    fgmask = fgbg.apply(frame)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel, iterations=2)
    fgmask = cv2.dilate(fgmask, kernel, iterations=2)
    fgmask = cv2.GaussianBlur(fgmask, (5, 5), 0)

    _, binary_mask = cv2.threshold(fgmask, 180, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    clean_mask = np.zeros_like(binary_mask)
    for cnt in contours:
        if cv2.contourArea(cnt) > 500:  
            cv2.drawContours(clean_mask, [cnt], -1, 255, -1)

    alpha = clean_mask.astype(float) / 255.0
    alpha = cv2.GaussianBlur(alpha, (15, 15), 0)
    alpha = np.expand_dims(alpha, axis=2)

    foreground = frame.astype(float) * alpha
    background = bg_img.astype(float) * (1 - alpha)
    output_frame = cv2.add(foreground, background).astype(np.uint8)

    out.write(output_frame)

In [8]:
cap.release()
out.release()

print("Final video saved as:", output_path)

Final video saved as: final_output.mp4
